## Установка pandas

In [ ]:
!pip3 install pandas

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Чтение данных и фильтрация таблиц

In [ ]:
base = 'drive/MyDrive/data'
user = pd.read_csv(base+ '/user.csv')
user.head()

,id,gender,birthday,grade,region,last_visited
0,1,М,02.11.2007,9.0,Город Санкт-Петербург,23.08.2023 03:04:14
1,2,Ж,29.01.2009,8.0,Город Москва,07.09.2023 09:21:35
2,3,М,09.05.2007,NaN,Город Санкт-Петербург,23.08.2023 18:32:18
3,4,Ж,27.09.2007,9.0,Город Москва,18.08.2023 07:34:11
4,5,Ж,04.08.2007,NaN,Город Москва,02.09.2023 18:31:30


### Задание 1

Сколько в таблице `user` представлено учеников, родившихся в сентябре?

In [ ]:
user['birthday'] = pd.to_datetime(user['birthday'], format='%d.%m.%Y')
print(len(user[user['birthday'].dt.month == 9]))

73


### Задание 2

In [ ]:
user['last_visited'] = pd.to_datetime(user['last_visited'], format='%d.%m.%Y %H:%M:%S')
print(len(user[(user['last_visited'].dt.month == 9) & (user['last_visited'].dt.day <= 14)]))

377


Какое количество учеников в последний раз посещали платформу в первые $14$ дней сентября?

### Задание 3

На сколько в данных больше девятиклассников, чем девятиклассниц?

In [ ]:
nine_grade = user[user['grade'] == 9]
print(len(nine_grade[nine_grade['gender'] == "М"]) - len(nine_grade[nine_grade['gender'] == "Ж"]))

5


### Задание 4

Какой процент учеников, у которых указан класс, обучается в $9$-м классе?

Ответ округлите до целых.

In [ ]:
grade_users = user[user['grade'].notna()]
print((len(nine_grade) / len(grade_users) * 100))

23.706377858002405


### Задание 5

Сколько в данных представлено учеников из регионов, в названиях которых в качестве подстроки содержится слово «город»?

При расчёте статистики приведите названия регионов к строчному написанию.

In [ ]:
gorod_users = user[user['region'].str.lower() \
                   .str.contains('город')]
print(len(gorod_users))

765


## Группировка, агрегация и соединение таблиц

In [ ]:
course = pd.read_csv(base + '/course.csv')
course.head()

,id,subject,date_started,max_score
0,1,Химия,2023-04-04,108.0
1,2,Физика,2023-01-31,137.0
2,3,Биология,2023-09-14,NaN
3,4,Химия,2023-03-31,118.0
4,5,Физика,2023-11-30,NaN


In [ ]:
user_course_progress = pd.read_csv(base+ '/user_course_progress.csv')
user_course_progress.head()

,user_id,course_id,progress
0,392,1,31.68
1,217,1,97.53
2,843,1,81.50
3,417,1,15.92
4,605,1,103.31


**Относительным прогрессом** будем называть отношение суммарного числа баллов, которые ученик набрал за курс (колонка `progress` в таблице `user_course_progress`), к максимальному количеству баллов, которые ученик в принципе мог набрать за данный курс (колонка `max_progress` в таблице `course`).

С помощью представленных данных найдите ответ на следующий вопрос: в курсах какого предмета больше всего учеников, чей относительный прогресс не меньше $0.8$?

В рамках следующих заданий вам будет предложено, действуя поэтапно, найти ответ на этот вопрос.

### Задание 1

In [ ]:
user_course_merged = user_course_progress \
                     .merge(course, left_on='course_id', right_on='id', how='inner') \
                     .drop(columns='id')
print(user_course_merged.head()                )

print(len(user_course_merged[user_course_merged['subject'] == 'Химия']))

   user_id  course_id  progress subject date_started  max_score
0      392          1     31.68   Химия   2023-04-04      108.0
1      217          1     97.53   Химия   2023-04-04      108.0
2      843          1     81.50   Химия   2023-04-04      108.0
3      417          1     15.92   Химия   2023-04-04      108.0
4      605          1    103.31   Химия   2023-04-04      108.0
678


Произведите соединение таблиц `course` и `user_course_progress`.

В качестве ответа укажите количество строчек итоговой таблицы, в которых представлены ученики курсов по химии.

### Задание 2

Обратите внимание, что у части курсов нет значения в колонке `max_score`. Уберите из полученной в предыдущем задании таблицы все строчки, в которых пропущено значение колонки `max_score`.

В качестве ответа укажите количество строчек в итоговой таблице.

In [ ]:
user_course_merged = user_course_merged.dropna(subset='max_score')
print(len(user_course_merged))

1031


### Задание 3

Добавьте в полученную в предыдущем задании таблицу колонку `relative_progress`, в которой для каждого ученика будет храниться значение его относительного прогресса в конкретном курсе.

В качестве ответа укажите среднее значение относительного прогресса в таблице. Ответ округлите до двух знаков после запятой с помощью функции `round`, которая встроена в Series.

In [ ]:
user_course_merged['relative_progress'] = pd.Series(
    user_course_merged['progress'] / user_course_merged['max_score'])
print(user_course_merged['relative_progress'].mean().round(2))

0.35


### Задание 4

Оставьте в полученной в предыдущем задании таблице только те строчки, в которых значение относительного прогресса не меньше $0.8$.

В качестве ответа укажите число строчек в получившейся таблице.

In [ ]:
top_progressed_uc = user_course_merged[user_course_merged['relative_progress'] >= 0.8]
len(top_progressed_uc)

159

### Задание 5

Произведите группировку строчек таблицы по предметам и посчитайте, сколько учеников с относительным прогрессом не меньше $0.8$ оказалось в каждом из предметов.

В качестве ответа заполните таблицу, в которой каждому предмету сопоставлено число таких учеников.

Если один и тот же ученик учился в разных курсах одного и того же предмета и в каждом из этих курсов достиг относительного прогресса не менее $0.8$, при подсчёте учтите его несколько раз.

In [ ]:
subject_grouped = top_progressed_uc.groupby('subject', as_index=False).agg(subject_count=('subject', 'size'))
subject_grouped

,subject,subject_count
0,Математика,30
1,Физика,24
2,Химия,105


### Задание 6

В курсах какого предмета больше всего учеников, достигших относительного прогресса не менее $0.8$?